# Simple and fast Stable Diffusion Colab
Github: [SlavyanDesu/sd-webui-colab](https://github.com/SlavyanDesu/sd-webui-colab)

In [ ]:
#@markdown # Use Google Drive for SD Web UI (Optional)
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@markdown # Clone Stable Diffusion repo
from IPython.display import clear_output
import os
import time

if not os.path.exists("/content/gdrive/MyDrive/"):
  print("Google drive not connected, using Colab instead...")
  !mkdir -p /content/gdrive/MyDrive/

time.sleep(3)
%cd /content/gdrive/MyDrive
!git clone --depth 1 -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui

%cd /content/gdrive/MyDrive/stable-diffusion-webui/
!git reset --hard
time.sleep(1)
!rm webui.sh

!git pull
clear_output()
print("Done! Proceed to next step.")

In [ ]:
#@markdown # Installing Requirements
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274642/google-perftools_2.5-2.2ubuntu3_all.deb
!wget http://launchpadlibrarian.net/367274648/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget http://launchpadlibrarian.net/367274647/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2
%pip install -q --upgrade fastapi==0.90.1
%pip install -q --upgrade xformers

clear_output()
print("Done! Proceed to next step.")

In [ ]:
#@markdown # Model Download
Model_Name = "SD v1.5" #@param ["SD v1.5", "ChilloutMix", "Grapefruit v4.1", "Counterfeit v2.5"]

#@markdown Or
Model_Link = "" #@param {type:"string"}
Model_Filename = "" #@param {type:"string"}
safetensors = False #@param {type:"boolean"}

pth = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Stable-diffusion"
ext_path = "/content/gdrive/MyDrive/stable-diffusion-webui/extensions"
vae_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/VAE"
lora_path = "/content/gdrive/MyDrive/stable-diffusion-webui/models/Lora"

if Model_Link != "":
  time.sleep(1)
  %cd $pth
  clear_output()
  if not safetensors:
    modelname = Model_Name + ".ckpt"
  else:
    modelname = Model_Name + ".safetensors"
  time.sleep(1)
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $Model_Link -d $pth -o $modelname
  if os.path.exists(f'{pth}/{modelname}'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Invalid link, please check the link...")
elif Model_Name == "SD v1.5":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -d $pth -o v1-5-pruned-emaonly.ckpt
  if os.path.exists(f'{pth}/v1-5-pruned-emaonly.ckpt'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")
elif Model_Name == "ChilloutMix":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/11745 -d $pth -o chilloutmix_NiPrunedFp32Fix.safetensors
  if os.path.exists(f'{pth}/chilloutmix_NiPrunedFp32Fix.safetensors'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")
elif Model_Name == "Grapefruit v4.1":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16619 -d $pth -o grapefruitHentaiModel_grapefruitv41.safetensors
  if os.path.exists(f'{pth}/grapefruitHentaiModel_grapefruitv41.safetensors'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")
elif Model_Name == "Counterfeit v2.5":
  time.sleep(1)
  %cd $pth
  clear_output()
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/7425 -d $pth -o CounterfeitV25_25.safetensors
  if os.path.exists(f'{pth}/CounterfeitV25_25.safetensors'):
    clear_output()
    print("Model downloaded, using the trained model...")
    print("Done! Proceed to next step.")
  else:
    print("Something went wrong...")


In [ ]:
#@markdown # Downloading Extensions
!git clone https://github.com/SlavyanDesu/sd-webui-tunnels $ext_path/sd-webui-tunnels
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete $ext_path/a1111-sd-webui-tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git $ext_path/ultimate-upscale-for-automatic1111
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser.git $ext_path/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/sd-civitai-browser $ext_path/sd-civitai-browser
!git clone https://github.com/catppuccin/stable-diffusion-webui $ext_path/catppuccin-theme
clear_output()
print("Done! Proceed to next step.")

In [ ]:
#@markdown # Downloading VAE and Embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt -d $vae_path -o kl-f8-anime.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d $vae_path -o kl-f8-anime2.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt -d $vae_path -o Counterfeit-V2.5.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -d $vae_path -o orangemix.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d $vae_path -o vae-ft-mse-840000-ema-pruned.ckpt

import shutil
shutil.rmtree('/content/gdrive/MyDrive/stable-diffusion-webui/embeddings')
%cd /content/gdrive/MyDrive/stable-diffusion-webui
!git clone https://huggingface.co/SlavyanDesu/embeddings
clear_output()
print("Done! Proceed to next step.")

In [ ]:
#@markdown # Downloading LORA for KoreanDollLikeness (Optional)
#@markdown Somehow the author of KoreanDollLikeness deleted his LORA on Civitai, so I reuploaded it.
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/SlavyanDesu/KDLikenessv15/resolve/main/koreanDollLikeness_v15.safetensors -d $lora_path -o koreanDollLikeness_v15.safetensors
clear_output()
print("Done! Proceed to next step.")

In [ ]:
#@markdown # Start Stable Diffusion
!COMMANDLINE_ARGS="--share --xformers --disable-safe-unpickle --no-half-vae --enable-insecure-extension-access --remotemoe" REQS_FILE="requirements.txt" python launch.py